In [2]:
import os
print(os.listdir())

['dukest1resurvey2009.json', 'smallirebysave.res', 'dukest1resurvey2009.tscn', '0_917647_0_917647_0_917647_0_000000.material', 'dukest1resurvey2009.3d', 'parse3D_work.ipynb', 'picturepalace2.gltf', 'LambTrap-drawnup-1.png', 'LargePot', '0_980392_0_713725_0_003922_0_000000.material', 'dukest1resurvey2009.gltf', 'scanimagefloor.material', 'skirwith', 'LambTrap-drawnup-1.png.import', '0_498039_0_498039_0_498039_0_000000.material', 'picturehouse.glb.import', '0_615686_0_811765_0_929412_0_000000.material', 'picturepalace.gltf', '.ipynb_checkpoints', 'wingform', '0_972549_0_529412_0_003922_0_000000.material', '0_647059_0_647059_0_647059_0_000000.material', '__pycache__', '0_729412_0_250980_0_105882_0_000000.material', 'picturepalace.gltf.import', 'Ireby', 'picturehouse.glb', 'parse3ddmp.py', 'dukest1resurvey2009.gltf.import', 'DukeStResurvey-drawnup-p3.jpg.import', 'picturepalace2.gltf.import', 'notes.txt', 'convertdmptotunnelvrjson.py', 'DukeStResurvey-drawnup-p3.jpg', '0_372549_0_654902_0_

In [7]:
from parse3ddmp import DMP3d

fname = "skirwith/skirwith-cave.3d"
dump3dexe = 'dump3d'

dstream = os.popen("%s -d %s" % (dump3dexe, fname))
lines = dstream.readlines()
dmp3d = DMP3d(lines)


In [8]:
dmp3d.cs

'+init=epsg:27700 +no_defs'

In [14]:
import pyproj
proj = pyproj.Proj(dmp3d.cs)
proj(370971.19, 473746.38, inverse=True)


(-2.4445880673413165, 54.15853521413708)

In [11]:
leglines = [line  for line in dmp3d.lines  if line[0] != line[1] and "SURFACE" not in line[4]]
len(leglines)
leg = leglines[0]
leg

(P3(370971.19, 473746.38, 259.43),
 P3(370969.28, 473746.58, 259.1),
 'skirwith-cave.skirwith-upper-entrance-1',
 'NORMAL',
 [],
 datetime.datetime(2021, 4, 12, 0, 0))

In [12]:
dir(dmp3d)

['GetXSectionFacts',
 'GetXSectionQuadHedron',
 'GetXSectionQuadHedronP',
 'PassagesLength',
 'PassagesVolume',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'cs',
 'datenumeric',
 'headdate',
 'lines',
 'nmapnodes',
 'nodepmap',
 'nodes',
 'printreport',
 'title',
 'xsects']

In [1]:
from parse3ddmp import DMP3d
import os

dump3dexe = '"C:\\Program Files (x86)\\Survex\\dump3d.exe"'
fname = "dukest1resurvey2009.3d"
fjson = "dukest1resurvey2009.json"
fname = "cusseypot\\Cussey_Pot_Master.3d"
fjson = "cusseypot\\Cussey_Pot_Master.json"
fname = "cusseypot\\skirwith-cave.3d"

dstream = os.popen("%s -d %s" % (dump3dexe, fname))
lines = dstream.readlines()
dmp3d = DMP3d(lines)

"""
    self.nodepmap = { }   # { p: ([names], [flagsconsolidated] }  
    self.nodes = [ ]      # [ (p, name, [flags]) ]   (not strictly necessary, except to get entrance name before flagconsolidation)
    self.lines = [ ]      # [ (prevp, p, name, style, [flags], date) ]
    self.xsects = [ [ ] ] # [ [ (name, lrud) ] ]
    self.nmapnodes = { }  # { name: p } (derived from nodepmap, needed for xsects)
""";

In [2]:
from parse3ddmp import P3
import json

leglines = [line  for line in dmp3d.lines  if line[0] != line[1] and "SURFACE" not in line[4]]
xsects = [xsect  for xsect in dmp3d.xsects  if len(xsect) >= 2]

assert not dmp3d.cs, "We need to convert from absolute coords in this case"

# points as triples
stationpoints = set()
stationpoints.update(line[0]  for line in leglines)
stationpoints.update(line[1]  for line in leglines)
stationpoints = list(stationpoints)
stationpoints.sort(key=lambda X: (X[2], X[0], X[1]))

# centralize the sketch
def rangerat(ps, i, maxfac):
    return (1-maxfac)*min(p[i]  for p in ps) + maxfac*max(p[i]  for p in ps)
svxp0 = P3(rangerat(stationpoints, 0, 0.5), rangerat(stationpoints, 1, 0.5), rangerat(stationpoints, 2, 1.0))

stationpointsdict = dict((p, i)  for i, p in enumerate(stationpoints))
stationpointscoords = sum((tuple(stationpoint - svxp0)  for stationpoint in stationpoints), ())
stationpointsnames = [ dmp3d.nodepmap[stationpoint][0][0]  for stationpoint in stationpoints ]
legsconnections = sum(((stationpointsdict[legline[0]], stationpointsdict[legline[1]])  for legline in leglines), ())
legsconnections = sum(((stationpointsdict[legline[0]], stationpointsdict[legline[1]])  for legline in leglines), ())
legsstyles = [ legline[3]  for legline in leglines ]

xsectgps = [ ]
for xsectseq in xsects:
    xsectpoints = [dmp3d.nmapnodes[xs[0]]  for xs in xsectseq]
    xsectindexes = [ stationpointsdict[xsectpoint]  for xsectpoint in xsectpoints ]
    xsectrightvecs = [ ]
    for i in range(len(xsectpoints)):
        pback = xsectpoints[max(0, i-1)]
        p = xsectpoints[i]
        pfore = xsectpoints[min(len(xsectpoints)-1, i+1)]
        vback = P3.ZNorm(P3(p.x - pback.x, p.y - pback.y, 0.0))
        vfore = P3.ZNorm(P3(pfore.x - p.x, pfore.y - p.y, 0.0))
        xsectplanevec = P3.ZNorm(vback + vfore)
        xsectrightvecs.append(xsectplanevec[1])
        xsectrightvecs.append(-xsectplanevec[0])
    xsectrightvecs
    xsectlruds = sum((xs[1]  for xs in xsectseq), ())
    xsectlruds
    xsectgps.append({ "xsectindexes":xsectindexes, "xsectrightvecs":xsectrightvecs, "xsectlruds":xsectlruds })
xsectgps

jrec = { "stationpointscoords": stationpointscoords,
         "stationpointsnames":  stationpointsnames, 
         "legsconnections":     legsconnections,
         "legsstyles":          legsstyles, 
         "xsectgps":            xsectgps
       }
print(" ".join("%s:%d"%(k, len(v))  for k, v in jrec.items()))
open(fjson, "w").write(json.dumps(jrec))

169854